In [28]:
import socket
import struct

Использование socket вы можете нагуглить и без меня, ниже пример тестового кода. Подобное можно встретить на просторах интернета в несчетных количествах(а это значит, что нужно рассмотреть возможность скопипастить код с, например, stackoverflow).

Системные порты кончаются на 1024ом, их вам система просто так не даст.
Поэтому нужно выбирать порт 1024+, но не более 65000, где-то там значение 2^16 (16 битная адресация все таки).

In [237]:
PORT = 64999

In [238]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(("localhost", PORT))
server_socket.listen(1) #мы ожидаем 1ого клиента

In [239]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(("localhost", PORT))

In [240]:
client, addr = server_socket.accept()

In [241]:
client_socket.send("Hello, server!".encode())

14

In [242]:
data = client.recv(1024)
print(data.decode())

Hello, server!


Чтоб разобраться в том, что такое struct, нужно либо найти документацию(ха ха), либо воспользоваться dir + help.

In [243]:
# dir(struct)

In [244]:
# help(struct)

\#help(struct)
Вот особенно интересный кусок __хелпа__.

The optional first format char indicates byte order, size and alignment:
 * @: native order, size & alignment (default)
 * =: native order, std. size & alignment
 * <: little-endian, std. size & alignment
 * \>: big-endian, std. size & alignment
 * !: same as >

The remaining chars indicate types of args and must match exactly;
these can be preceded by a decimal repeat count:
 * x: pad byte (no data); c:char; b:signed byte; B:unsigned byte;
 * ?: _Bool (requires C99; if not available, char is used instead)
 * h:short; H:unsigned short; i:int; I:unsigned int;
 * l:long; L:unsigned long; f:float; d:double; e:half-float.
Special cases (preceding decimal count indicates length):
 * s:string (array of char); p: pascal string (with count byte).
Special cases (only available in native format):
 * n:ssize_t; N:size_t;
 * P:an integer type that is wide enough to hold a pointer.
Special case (not in native mode unless 'long long' in platform C):
 * q:long long; Q:unsigned long long
Whitespace between formats is ignored.

In [245]:
struct.pack('i', 1).hex()

'01000000'

In [246]:
struct.pack('ii', 1, 1).hex()

'0100000001000000'

In [247]:
struct.pack('i', 2**31-1).hex()

'ffffff7f'

In [248]:
struct.pack('I', 2**32-1).hex()

'ffffffff'

In [249]:
struct.pack('4s', 'aaaA'.encode())

b'aaaA'

Посмотрев выше можно сделать вывод, что в рамках данного курса не важно в какой именно форме будет байтовый массив. Достаточно принять тот факт, что struct.pack переводит в __какой-то__ формат.

In [250]:
struct.calcsize('ii')

8

In [251]:
client_socket.send(struct.pack('ii', 1, 2))

8

In [252]:
data = client.recv(1024)
print(struct.unpack('ii', data))

(1, 2)


In [253]:
for i, in struct.iter_unpack('i', struct.pack('iiii', 1, 2, 3, 4)):
    print(i)

1
2
3
4


Ниже представлен код, в котором вам самим было бы неплохо разобраться, удачи)

In [254]:
string = '!'*20000 # много много символов
client_socket.send(struct.pack(f'I{len(string)}s', len(string), string.encode()))

data = client.recv(1024)

recv_size0, = struct.iter_unpack('I', data).__iter__().__next__()
recv_size1, = struct.unpack('I', data[:struct.calcsize('I')])
print(f'{recv_size0} == {recv_size1}')

expected_max_string_size = 1024 - struct.calcsize('I') #то что здесь magic number, не значит, что так надо делать) 
current_extracting_size = recv_size0 if recv_size0 < expected_max_string_size else expected_max_string_size

_, string_buff = struct.unpack(f'I{current_extracting_size}s', data)

recv_size0 -= current_extracting_size
expected_max_string_size = 1024

while recv_size0 > 0:
    current_extracting_size = recv_size0 if recv_size0 < expected_max_string_size else expected_max_string_size
    
    data = client.recv(1024)
    string_buff += struct.unpack(f'{current_extracting_size}s', data)[0]
    
    recv_size0 -= current_extracting_size

recv_string = string_buff.decode()

print(f'{recv_string[-1:-20:-1]}... {len(recv_string)}')

20000 == 20000
!!!!!!!!!!!!!!!!!!!... 20000


!!!!!


Не забываем закрыть соединения. Напоминаю про ключевое слово __with__.

(2,)


В рамках второй лабораторной у вас должна возникнуть проблема, состоящая в следующем:
* вы устанавливаете соединение;
* отправляете данные в нескольких пакетах(ок, sendall отправит все разом, но читать вы будете все равно буфером ограниченного размера);
* вы прочитали N пакетов, но не смогли остановиться...
* socket.socket.recv блокирует поток в ожидании данных и программа зависает.

Как с этим можно справиться? можно спросить меня на паре или прочитать ниже:
* После того как вы досылаете последний пакет, закрывайте соединение(см. with). Читать из закрытого соединения нельзя, что значит socket.socket.recv прочитает 0 байт и блокировать ничего не будет;
* Отсылать маркер(пакет определенного размера/содержания) после завершения отправки всех данных. А на получении маркера прекращать читать сокет;
* Сначала отправить размер данных, а потом читать именно такой размер(код выше, но это не значит, что его нужно копипастить).